In [1]:
import pandas as pd
import torch
from torch import nn

processedpath='../data/processed_data/'

In [2]:
player_engagement_with_info = pd.read_pickle(processedpath+'player_engagement_with_info.pkl')

In [3]:
player_engagement_with_info.drop(['date','playerName','rosterTeamName','rosterTeamName','gameTeamName','oppName','awardId'],axis=1,inplace=True)
player_engagement_with_info.drop_duplicates(inplace=True)

In [4]:
# grouping columns
target_cols=[
    'target1',
    'target2',
    'target3',
    'target4',
    'targetAvg'
]

index_cols=[
    'dailyDataDate',
    'engagementMetricsDate',
    'year',
    'month'
]

date_info_cols=[
    'inSeason',
    'seasonPart',
    'days_since_preseason_start',
    'days_since_preseason_end',
    'days_since_postseason_start',
    'days_since_postseason_end',
    'days_since_regular_season_start_1st',       
    'days_since_regular_season_end_2nd',       
    'days_since_regular_season_end_1st',       
    'days_since_regular_season_start_2nd',       
    'days_since_allStar'
]

player_info_cols=[
    'playerId',
    'birthCity',
    'birthStateProvince',
    'birthCountry',
    'primaryPositionName',
    'age',
    'days_since_debut'
]

roster_cols=[
    'rosterStatus',
    'rosterTeamId',
    'rosterTeamLocation',      
    'rosterTeamLeagueId',    
    'rosterDivisionId'
]

players_games_agg_cols=[
    'numGames',
    'numTeams',       
    'runsScored',       
    'homeRuns',       
    'strikeOuts',       
    'baseOnBalls',       
    'hits',       
    'hitByPitch',       
    'atBats',       
    'caughtStealing',       
    'stolenBases',       
    'groundIntoDoublePlay',       
    'groundIntoTriplePlay',       
    'plateAppearances',       
    'totalBases',       
    'rbi',       
    'leftOnBase',       
    'sacBunts',       
    'sacFlies',       
    'gamesStartedPitching',       
    'runsPitching',       
    'homeRunsPitching',       
    'strikeOutsPitching',       
    'baseOnBallsPitching',       
    'hitsPitching',       
    'inningsPitchedAsFrac',       
    'earnedRuns',       
    'battersFaced',       
    'saves',       
    'blownSaves',       
    'pitchingGameScore',       
    'noHitter'
]

games_aggg_cols=[
    'gameTeamLocation',        
    'gameTeamLeagueId',       
    'gameDivisionId',       
    'gameType',        
    'gameTeamId',       
    'oppId',       
    'rivalry_score',       
    'numGamesTeam'       
]

games_result_cols=[
    'winsTeam',       
    'lossesTeam',       
    'runsScoredTeam',       
    'runsAllowedTeam'       
]

transaction_cols=[
    'typeCode',        
    'transaction_category'
]

award_cols=[
    'num_awards',       
    'max_award_streak',       
    'min_award_streak',       
    'mean_award_streak',       
    'end_streak'       
]

standing_cols=[
    'divisionRankTeam',
    'leagueRankTeam',       
    'wildCardRankTeam',       
    'winPctTeam',       
    'winStreakTeam',       
    'lossStreakTeam'
]

In [11]:
for i in os.listdir(processedpath):
    if '.pkl' in i:
        globals()[i[:-4]] = pd.read_pickle(processedpath + i)

In [40]:
game_schedule=team_games[['dailyDataDate','season','gameType','teamId','oppId','isHomeTeam','rivalry_score']].drop_duplicates()

In [42]:
players_games_agg_cols

['numGames',
 'numTeams',
 'runsScored',
 'homeRuns',
 'strikeOuts',
 'baseOnBalls',
 'hits',
 'hitByPitch',
 'atBats',
 'caughtStealing',
 'stolenBases',
 'groundIntoDoublePlay',
 'groundIntoTriplePlay',
 'plateAppearances',
 'totalBases',
 'rbi',
 'leftOnBase',
 'sacBunts',
 'sacFlies',
 'gamesStartedPitching',
 'runsPitching',
 'homeRunsPitching',
 'strikeOutsPitching',
 'baseOnBallsPitching',
 'hitsPitching',
 'inningsPitchedAsFrac',
 'earnedRuns',
 'battersFaced',
 'saves',
 'blownSaves',
 'pitchingGameScore',
 'noHitter']

In [32]:
team_games.gameNumber


0        1
1        1
2        1
3        1
4        1
        ..
12523    1
12524    1
12525    1
12526    1
12527    1
Name: gameNumber, Length: 12528, dtype: int64

## Organic increase (Trend) vs. From performance

In [6]:
player_engagement_with_info.columns

Index(['dailyDataDate', 'engagementMetricsDate', 'playerId', 'target1',
       'target2', 'target3', 'target4', 'targetAvg', 'year', 'month',
       'inSeason', 'seasonPart', 'days_since_preseason_start',
       'days_since_preseason_end', 'days_since_postseason_start',
       'days_since_postseason_end', 'days_since_regular_season_start_1st',
       'days_since_regular_season_end_2nd',
       'days_since_regular_season_end_1st',
       'days_since_regular_season_start_2nd', 'days_since_allStar',
       'birthCity', 'birthStateProvince', 'birthCountry',
       'primaryPositionName', 'age', 'days_since_debut', 'rosterStatus',
       'rosterTeamId', 'rosterTeamLocation', 'rosterTeamLeagueId',
       'rosterDivisionId', 'numGames', 'numTeams', 'gameTeamId', 'runsScored',
       'homeRuns', 'strikeOuts', 'baseOnBalls', 'hits', 'hitByPitch', 'atBats',
       'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
       'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
     

In [5]:
target_df=player_engagement_with_info[target_cols]
target_df['median']=player_engagement_with_info.iloc[:,3:7].median(axis=1)

ipykernel_launcher:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
target_df.corr(method='spearman')

,target1,target2,target3,target4,targetAvg,median
target1,1.000000,0.708682,0.708998,0.616399,0.742040,0.764480
target2,0.708682,1.000000,0.655824,0.678315,0.948252,0.858723
target3,0.708998,0.655824,1.000000,0.596294,0.706980,0.737858
target4,0.616399,0.678315,0.596294,1.000000,0.819061,0.877892
targetAvg,0.742040,0.948252,0.706980,0.819061,1.000000,0.908149
median,0.764480,0.858723,0.737858,0.877892,0.908149,1.000000
